In [85]:
# Import all cars from opendbc

from opendbc.car import structs
from opendbc.car.values import PLATFORMS as TEST_PLATFORMS

# Example: add additional platforms/segments to test outside of commaCarSegments

EXTRA_SEGMENTS = {
  # "81dd9e9fe256c397/0000001f--97c42cf98d",  # Volkswagen ID.4 test route, new car port, not in public dataset
}

In [86]:
import random

from openpilot.tools.lib.logreader import LogReader
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database


MAX_SEGS_PER_PLATFORM = 3  # Increase this to search more segments

database = get_comma_car_segments_database()
TEST_SEGMENTS = []

print(f"Searching {len(TEST_PLATFORMS)} platforms")

for platform in TEST_PLATFORMS:
  if platform not in database:
    print(f"No segments available for {platform}")
    continue

  all_segments = database[platform]
  NUM_SEGMENTS = min(len(all_segments), MAX_SEGS_PER_PLATFORM)
  TEST_SEGMENTS.extend(random.sample(all_segments, NUM_SEGMENTS))

TEST_SEGMENTS.extend(EXTRA_SEGMENTS)

print(f"Searching {len(TEST_SEGMENTS)} segments")

Searching 221 platforms
No segments available for DODGE_DURANGO
No segments available for FORD_RANGER_MK2
No segments available for HOLDEN_ASTRA
No segments available for CADILLAC_ATS
No segments available for CHEVROLET_MALIBU
No segments available for CADILLAC_XT4
No segments available for CHEVROLET_VOLT_2019
No segments available for CHEVROLET_TRAVERSE
No segments available for GMC_YUKON
No segments available for HONDA_ODYSSEY_CHN
No segments available for HYUNDAI_KONA_2022
No segments available for HYUNDAI_NEXO_1ST_GEN
No segments available for GENESIS_GV70_ELECTRIFIED_1ST_GEN
No segments available for GENESIS_G80_2ND_GEN_FL
No segments available for RIVIAN_R1_GEN1
No segments available for SUBARU_FORESTER_HYBRID
No segments available for TESLA_MODEL_3
No segments available for TESLA_MODEL_Y
No segments available for TOYOTA_RAV4_PRIME
No segments available for TOYOTA_SIENNA_4TH_GEN
No segments available for LEXUS_LC_TSS2
No segments available for VOLKSWAGEN_CADDY_MK3
No segments ava

In [ ]:
from openpilot.tools.lib.logreader import LogReader, comma_car_segments_source
from tqdm.notebook import tqdm, tnrange

# Example search for CAN ignition messages
# Be careful when filtering by bus, account for odd harness arrangements on Honda/HKG

BUSES_TO_SEARCH = [0, 1, 2]

# Support for external Red Panda
EXTERNAL_PANDA_BUSES = [bus + 4 for bus in BUSES_TO_SEARCH]

MESSAGES_TO_FIND = {
  0x1F1: "GM CAN Ign",
  0x152: "Rivian CAN Ign",
  0x221: "Tesla 3/Y CAN Ign",
  0x9E:  "Mazda CAN Ign",
  0x3C0: "VW CAN Ign",
}

progress_bar = tnrange(len(TEST_SEGMENTS), desc="segments searched")

for segment in TEST_SEGMENTS:
  lr = LogReader(segment, sources=[comma_car_segments_source])
  CP = lr.first("carParams")
  if CP is None:
    progress_bar.update()
    continue

  can_packets = [msg for msg in lr if msg.which() == "can"]
  matched_messages = set()

  for packet in can_packets:
    for msg in packet.can:
      if msg.address in MESSAGES_TO_FIND and msg.src in (BUSES_TO_SEARCH + EXTERNAL_PANDA_BUSES):
        # print(msg)
        matched_messages.add(msg.address)

  if len(matched_messages) > 0:
    message_names = [MESSAGES_TO_FIND[message] for message in matched_messages]
    print(f"Match found: {segment:<45} {CP.carFingerprint:<38} {message_names}")

  progress_bar.update()


segments searched:   0%|          | 0/577 [00:00<?, ?it/s]

Match found: 0f53b336851e1384/2023-11-20--09-44-03/12/s    CHRYSLER PACIFICA HYBRID 2018          ['VW CAN Ign']
Match found: 7620ad20d3cefc64/2023-10-28--08-14-40/3/s     CHRYSLER PACIFICA HYBRID 2018          ['VW CAN Ign']
Match found: 00d247a9bb1f9196/2023-11-06--13-33-17/9/s     CHRYSLER PACIFICA HYBRID 2018          ['VW CAN Ign']
Match found: 120a432f63cb0de2/2023-10-30--20-01-34/1/s     CHRYSLER PACIFICA HYBRID 2019          ['VW CAN Ign']
Match found: b70b56b76a6217f2/2023-12-19--08-30-22/35/s    CHRYSLER PACIFICA HYBRID 2019          ['VW CAN Ign']
Match found: 97e388680a6716ed/2024-01-17--10-15-13/9/s     CHRYSLER PACIFICA HYBRID 2019          ['VW CAN Ign']
Match found: 2137b01aa0ca63f9/2024-01-06--22-06-14/70/s    CHRYSLER PACIFICA 2018                 ['VW CAN Ign']
Match found: 8fc6a1b72c8b1357/2023-11-06--07-50-05/8/s     CHRYSLER PACIFICA 2018                 ['VW CAN Ign']
Match found: 7e705eb5c27a49cc/2024-01-18--16-51-20/3/s     CHRYSLER PACIFICA 2018               